In [1]:
from bs4 import BeautifulSoup
import requests 
import re
import pandas as pd

# This is the second part of crawlling
### For each one of our GTD_ID's we would like to get additional info

In [2]:
# This function returnes the GTD_Data_Frame_Without_Duplicates as dataframe.

def load_csv():
    df = pd.read_csv('GTD_Data_Frame_Without_Duplicates.csv')
    return df

In [3]:
# A function which receives GTD_IDs and return a list that contains every link that needs to be visited.

def getLinksForSecondCrawl(GTD_IDS):
    baseUrl = "https://www.start.umd.edu/gtd/search/IncidentSummary.aspx?gtdid="
    urlList = []
    
    for id in GTD_IDS:
        urlList.append(baseUrl +str(id))
    
    return urlList

In [4]:
def getMoreInfo(urlLinkToCrawl):
    response = requests.get(urlLinkToCrawl)
    soup = BeautifulSoup(response.content, "html.parser")
    temptd =soup.find_all("td",class_="value")
    templabel = soup.find_all("td",class_="label")
    attackType = "Unknown"
    successfulAttack = "Unknown"
    hostages  = "Unknown"
    ransom = "Unknown"
    propertyDamage = "Unknown"
    suicideAttack = "Unknown"
    partOfMultipleIncident = "Unknown"
    extentPropertyDamage = "Unknown"
    valuePropertyDamage = "Unknown"
    count =0
    

    for i in range(0,len(templabel)):
        if templabel[i].text =="Type of Attack (more)":
            if count==0:
                attackType=temptd[i].text
                count+=1     
        if templabel[i].text == "Successful Attack? (more)":
            successfulAttack=temptd[i].text
        if templabel[i].text == "Hostages":
            hostages=temptd[i].text
        if templabel[i].text =="Ransom":
            ransom=temptd[i].text
        if templabel[i].text =="Property Damage":
            propertyDamage=temptd[i].text
        if templabel[i].text =="Suicide Attack?":
            suicideAttack=temptd[i].text
        if templabel[i].text =="Part of Multiple Incident?":
            partOfMultipleIncident=temptd[i].text
        if templabel[i].text=="Extent of Property Damage":
            extentPropertyDamage = temptd[i].text
        if templabel[i].text== "Value of Property Damage":
            valuePropertyDamage = temptd[i].text
    
    #######################################################################################################
    tempDiv =soup.find_all("div",class_="results")
    typeOfWeapon ="Unknown"
    subTypeOfWeapon="Unknown"
    rawWeaponData=[]
    count= 1
    for div in tempDiv:
        if count==4:
            for tr in div.find_all("tr",class_="dark-gray"):
                for td in tr.find_all("td"):
                    rawWeaponData.append(td.text)
        count+=1
    
    if(len(rawWeaponData) >= 2):
        typeOfWeapon = rawWeaponData.pop(0)
        subTypeOfWeapon = rawWeaponData.pop(0)
    
    if typeOfWeapon =='':
        typeOfWeaponfWeapon="Unknown"
    if subTypeOfWeapon=='':
         subTypeOfWeapon="Unknown"
               
    return attackType,successfulAttack,hostages,ransom,propertyDamage,suicideAttack,partOfMultipleIncident,typeOfWeapon,subTypeOfWeapon
 

In [5]:
def tryGetNewInfoSplit(linksForSecondCrawl,fromWhere,toWhere):
    attackTypeList=[]
    successfulAttackList=[]
    hostagesList=[]
    ransomList=[]
    propertyDamageList=[]
    suicideAttackList=[]
    partOfMultipleIncidentList=[]
    typeOfWeaponList=[]
    subTypeOfWeaponList=[]
    
    subLinks = linksForSecondCrawl[fromWhere:toWhere]
    
    for link in subLinks:
        attackType,successfulAttack,hostages,ransom,propertyDamage,suicideAttack,partOfMultipleIncident,typeOfWeapon,subTypeOfWeapon=getMoreInfo(link)
        attackTypeList.append(attackType)
        successfulAttackList.append(successfulAttack)
        hostagesList.append(hostages)
        ransomList.append(ransom)
        propertyDamageList.append(propertyDamage)
        suicideAttackList.append(suicideAttack)
        partOfMultipleIncidentList.append(partOfMultipleIncident)
        typeOfWeaponList.append(typeOfWeapon)
        subTypeOfWeaponList.append(subTypeOfWeapon)
    
    print("Part Done")
    return attackTypeList,successfulAttackList,hostagesList,ransomList,propertyDamageList,suicideAttackList,partOfMultipleIncidentList,typeOfWeaponList,subTypeOfWeaponList

In [6]:
def save_new_csv(attackTypeList,successfulAttackList,hostagesList,ransomList,propertyDamageList,suicideAttackList,partOfMultipleIncidentList,typeOfWeaponList,subTypeOfWeaponList):
    df = pd.DataFrame({"attackTypeList":attackTypeList,"successfulAttackList":successfulAttackList,"hostagesList":hostagesList,"ransomList":ransomList,"propertyDamageList":propertyDamageList,"suicideAttackList":suicideAttackList,"partOfMultipleIncidentList":partOfMultipleIncidentList,"typeOfWeaponList":typeOfWeaponList,"subTypeOfWeaponList":subTypeOfWeaponList})
    return df

###  RUN

In [9]:
# Empty lists 
attackTypeList = []
successfulAttackList = []
hostagesList = []
ransomList = []
propertyDamageList = []
suicideAttackList = [] 
partOfMultipleIncidentList = [] 
typeOfWeaponList = [] 
subTypeOfWeaponList = []

# Where to start and where to end (each iteration)
fromWhere = 0
toWhere = 1
offset = 1

# load the csv file and get the GTD_IDs, after it get the links that we need in order to crawl.
df = load_csv()
GTD_ID = df.GTD_ID
linksForSecondCrawl = getLinksForSecondCrawl(GTD_ID)

##### We didin't want to make the servers of www.start.umd.edu too busy because of us so we decided to get 5000 results for each time

In [14]:
def lahavsFirstCrawl():
    # Empty lists 
    attackTypeList = []
    successfulAttackList = []
    hostagesList = []
    ransomList = []
    propertyDamageList = []
    suicideAttackList = [] 
    partOfMultipleIncidentList = [] 
    typeOfWeaponList = [] 
    subTypeOfWeaponList = []

    # Where to start and where to end (each iteration)
    fromWhere = 0
    toWhere = 1000
    offset = 1000

    # load the csv file and get the GTD_IDs, after it get the links that we need in order to crawl.
    df = load_csv()
    GTD_ID = df.GTD_ID
    linksForSecondCrawl = getLinksForSecondCrawl(GTD_ID)

    for i in range(0,50):
        tempAttackTypeList,tempSuccessfulAttackList,tempHostagesList,tempRansomList,tempPropertyDamageList,tempSuicideAttackList,tempPartOfMultipleIncidentList,tempTypeOfWeaponList,tempSubTypeOfWeaponList = tryGetNewInfoSplit(linksForSecondCrawl,fromWhere,toWhere)
        attackTypeList.extend(tempAttackTypeList)
        successfulAttackList.extend(tempSuccessfulAttackList)
        hostagesList.extend(tempHostagesList)
        ransomList.extend(tempRansomList)
        propertyDamageList.extend(tempPropertyDamageList)
        suicideAttackList.extend(tempSuicideAttackList)
        partOfMultipleIncidentList.extend(tempPartOfMultipleIncidentList)
        typeOfWeaponList.extend(tempTypeOfWeaponList) 
        subTypeOfWeaponList.extend(tempSubTypeOfWeaponList)
        fromWhere += offset
        toWhere += offset
    
    df = save_new_csv(attackTypeList,successfulAttackList,hostagesList,ransomList,propertyDamageList,suicideAttackList,partOfMultipleIncidentList,typeOfWeaponList,subTypeOfWeaponList)
    df.to_csv('lahavs_first_crawl.csv')

In [15]:
def lahavsSecondCrawl():
    # Empty lists 
    attackTypeList = []
    successfulAttackList = []
    hostagesList = []
    ransomList = []
    propertyDamageList = []
    suicideAttackList = [] 
    partOfMultipleIncidentList = [] 
    typeOfWeaponList = [] 
    subTypeOfWeaponList = []

    # Where to start and where to end (each iteration)
    fromWhere = 50000
    toWhere = 51000
    offset = 1000

    # load the csv file and get the GTD_IDs, after it get the links that we need in order to crawl.
    df = load_csv()
    GTD_ID = df.GTD_ID
    linksForSecondCrawl = getLinksForSecondCrawl(GTD_ID)

    for i in range(0,50):
        tempAttackTypeList,tempSuccessfulAttackList,tempHostagesList,tempRansomList,tempPropertyDamageList,tempSuicideAttackList,tempPartOfMultipleIncidentList,tempTypeOfWeaponList,tempSubTypeOfWeaponList = tryGetNewInfoSplit(linksForSecondCrawl,fromWhere,toWhere)
        attackTypeList.extend(tempAttackTypeList)
        successfulAttackList.extend(tempSuccessfulAttackList)
        hostagesList.extend(tempHostagesList)
        ransomList.extend(tempRansomList)
        propertyDamageList.extend(tempPropertyDamageList)
        suicideAttackList.extend(tempSuicideAttackList)
        partOfMultipleIncidentList.extend(tempPartOfMultipleIncidentList)
        typeOfWeaponList.extend(tempTypeOfWeaponList) 
        subTypeOfWeaponList.extend(tempSubTypeOfWeaponList)
        fromWhere += offset
        toWhere += offset
    
    df = save_new_csv(attackTypeList,successfulAttackList,hostagesList,ransomList,propertyDamageList,suicideAttackList,partOfMultipleIncidentList,typeOfWeaponList,subTypeOfWeaponList)
    df.to_csv('lahavs_second_crawl.csv')

In [16]:
def ShelisFirstCrawl():
    # Empty lists 
    attackTypeList = []
    successfulAttackList = []
    hostagesList = []
    ransomList = []
    propertyDamageList = []
    suicideAttackList = [] 
    partOfMultipleIncidentList = [] 
    typeOfWeaponList = [] 
    subTypeOfWeaponList = []

    # Where to start and where to end (each iteration)
    fromWhere = 100000
    toWhere = 101000
    offset = 1000

    # load the csv file and get the GTD_IDs, after it get the links that we need in order to crawl.
    df = load_csv()
    GTD_ID = df.GTD_ID
    linksForSecondCrawl = getLinksForSecondCrawl(GTD_ID)

    for i in range(0,50):
        tempAttackTypeList,tempSuccessfulAttackList,tempHostagesList,tempRansomList,tempPropertyDamageList,tempSuicideAttackList,tempPartOfMultipleIncidentList,tempTypeOfWeaponList,tempSubTypeOfWeaponList = tryGetNewInfoSplit(linksForSecondCrawl,fromWhere,toWhere)
        attackTypeList.extend(tempAttackTypeList)
        successfulAttackList.extend(tempSuccessfulAttackList)
        hostagesList.extend(tempHostagesList)
        ransomList.extend(tempRansomList)
        propertyDamageList.extend(tempPropertyDamageList)
        suicideAttackList.extend(tempSuicideAttackList)
        partOfMultipleIncidentList.extend(tempPartOfMultipleIncidentList)
        typeOfWeaponList.extend(tempTypeOfWeaponList) 
        subTypeOfWeaponList.extend(tempSubTypeOfWeaponList)
        fromWhere += offset
        toWhere += offset
    
    df = save_new_csv(attackTypeList,successfulAttackList,hostagesList,ransomList,propertyDamageList,suicideAttackList,partOfMultipleIncidentList,typeOfWeaponList,subTypeOfWeaponList)
    df.to_csv('shelis_first_crawl.csv')

In [17]:
def ShelisFirstCrawl():
    # Empty lists 
    attackTypeList = []
    successfulAttackList = []
    hostagesList = []
    ransomList = []
    propertyDamageList = []
    suicideAttackList = [] 
    partOfMultipleIncidentList = [] 
    typeOfWeaponList = [] 
    subTypeOfWeaponList = []

    # Where to start and where to end (each iteration)
    fromWhere = 150000
    toWhere = 151000
    offset = 1000

    # load the csv file and get the GTD_IDs, after it get the links that we need in order to crawl.
    df = load_csv()
    GTD_ID = df.GTD_ID
    linksForSecondCrawl = getLinksForSecondCrawl(GTD_ID)

    for i in range(0,50):
        tempAttackTypeList,tempSuccessfulAttackList,tempHostagesList,tempRansomList,tempPropertyDamageList,tempSuicideAttackList,tempPartOfMultipleIncidentList,tempTypeOfWeaponList,tempSubTypeOfWeaponList = tryGetNewInfoSplit(linksForSecondCrawl,fromWhere,toWhere)
        attackTypeList.extend(tempAttackTypeList)
        successfulAttackList.extend(tempSuccessfulAttackList)
        hostagesList.extend(tempHostagesList)
        ransomList.extend(tempRansomList)
        propertyDamageList.extend(tempPropertyDamageList)
        suicideAttackList.extend(tempSuicideAttackList)
        partOfMultipleIncidentList.extend(tempPartOfMultipleIncidentList)
        typeOfWeaponList.extend(tempTypeOfWeaponList) 
        subTypeOfWeaponList.extend(tempSubTypeOfWeaponList)
        fromWhere += offset
        toWhere += offset
    
    df = save_new_csv(attackTypeList,successfulAttackList,hostagesList,ransomList,propertyDamageList,suicideAttackList,partOfMultipleIncidentList,typeOfWeaponList,subTypeOfWeaponList)
    df.to_csv('shelis_second_crawl.csv')

##### Last crawl (the rest of the data)

In [ ]:
def lastCrawl():
    # Empty lists 
    attackTypeList = []
    successfulAttackList = []
    hostagesList = []
    ransomList = []
    propertyDamageList = []
    suicideAttackList = [] 
    partOfMultipleIncidentList = [] 
    typeOfWeaponList = [] 
    subTypeOfWeaponList = []

    # Where to start and where to end (each iteration)
    fromWhere = 200000
    toWhere = 201183
    offset = 1000

    # load the csv file and get the GTD_IDs, after it get the links that we need in order to crawl.
    df = load_csv()
    GTD_ID = df.GTD_ID
    linksForSecondCrawl = getLinksForSecondCrawl(GTD_ID)

    tempAttackTypeList,tempSuccessfulAttackList,tempHostagesList,tempRansomList,tempPropertyDamageList,tempSuicideAttackList,tempPartOfMultipleIncidentList,tempTypeOfWeaponList,tempSubTypeOfWeaponList = tryGetNewInfoSplit(linksForSecondCrawl,fromWhere,toWhere)
    attackTypeList.extend(tempAttackTypeList)
    successfulAttackList.extend(tempSuccessfulAttackList)
    hostagesList.extend(tempHostagesList)
    ransomList.extend(tempRansomList)
    propertyDamageList.extend(tempPropertyDamageList)
    suicideAttackList.extend(tempSuicideAttackList)
    partOfMultipleIncidentList.extend(tempPartOfMultipleIncidentList)
    typeOfWeaponList.extend(tempTypeOfWeaponList) 
    subTypeOfWeaponList.extend(tempSubTypeOfWeaponList)
    fromWhere += offset
    toWhere += offset
    
    df = save_new_csv(attackTypeList,successfulAttackList,hostagesList,ransomList,propertyDamageList,suicideAttackList,partOfMultipleIncidentList,typeOfWeaponList,subTypeOfWeaponList)
    df.to_csv('lastCrawl_second_crawl.csv')

##### save all of this into data frame

In [ ]:
lahavsFirstCrawl()

###### Save the dataframe to CSV file

Part Done
Part Done
Part Done
Part Done
Part Done
Part Done
Part Done
Part Done
Part Done
Part Done
